In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option("display.max_columns", None)
# pd.
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Reading data

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")

In [ ]:
print(len(train_df))

In [ ]:
train_df.head(5)

In [ ]:
Summary = pd.DataFrame(train_df.dtypes, columns = ['Dtype'])
Summary['null'] = train_df.isnull().sum()
Summary['max'] = train_df.max()
Summary['min'] = train_df.min()
Summary['first'] = train_df.iloc[0, :]
Summary['second'] = train_df.iloc[1, :]
Summary['third'] = train_df.iloc[2, :]

In [ ]:
Summary

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

In [ ]:
y = train_df["target"]
train_df = train_df.drop(["target", "id"], axis = 1)

In [ ]:
test_df = test_df.drop("id", axis = 1)

In [ ]:
train_df["mean"] = train_df.mean(axis = 1)
train_df["med"] = train_df.median(axis = 1)
train_df["std"] = train_df.std(axis = 1)
train_df["skew"] = train_df.skew(axis = 1)

test_df["mean"] = test_df.mean(axis = 1)
test_df["med"] = test_df.median(axis = 1)
test_df["std"] = test_df.std(axis = 1)
test_df["skew"] = test_df.skew(axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df,y, test_size = 0.20, stratify = y, random_state = 123)

In [ ]:
len(X_train)
len(X_test)

In [ ]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

In [ ]:
lr = LogisticRegression(random_state=123, C = 0.0001, penalty = 'l2')
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict_proba(X_test)
y_pred = y_pred[:, 1]

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(auc(fpr, tpr))
# The score is without adding features, 0.7469048481960304
# The score is with exrta features 0.746916605626198
# using just 20 percent data for testing, cause yeah, the more the better 0.7469685919166085
# score when used C = 0.005, 0.7469690275350676
# score when used C = 0.001 0.746970662771196
# score when used c = 0.0001 0.7469809539794716
# score when used skew as a feature as well 0.7470002189835658

In [ ]:
test_df = scaler.transform(test_df)

In [ ]:
y_res = lr.predict_proba(test_df)
y_res = y_res[:, 1]

In [ ]:
sub["target"] = y_res
sub.to_csv("lr1.csv", index=False)
sub.head(10)